In [1]:
from selenium import webdriver
import time
import warnings
from bs4 import BeautifulSoup
from selenium.common.exceptions import ElementClickInterceptedException
import pandas as pd

warnings.filterwarnings('ignore')

## Path to chrome driver. it should match same version as that of chrome browser.
## Download chrome driver from this location : https://chromedriver.chromium.org/downloads

DRIVER = '/home/sunny/Downloads/chromedriver_linux64/chromedriver' 

In [2]:
options = webdriver.ChromeOptions() # define options

## Comment below line if you want to see browser opening and url loading
## headless means that browser window won't be shown.
## Below we have set timer for page to load to 60 seconds. You can decide that timer by commenting this line.
options.add_argument("headless") 

driver = webdriver.Chrome(DRIVER, options=options)

## setting screen width and height in case "headless" statement is commented to see browser working.
driver.set_window_size(width=1700, height=950) 

In [3]:
def parse_table(page_source):
    ## Parsing Source Code of the Page
    parser = BeautifulSoup(page_source,"html")
    ## First Table
    first_table = parser.find_all("table")[0]
    ## Parsing Header Data
    header = first_table.find_all("thead")[0]

    header_data = []
    for cell in header.find_all("th"):
        header_data.append(cell.getText())

    ## Parsing Individual Row Data
    df_body = []
    for row in first_table.find_all("tr"):
        row_data = []
        for cell in row.find_all("td"):
            cell_data = cell.getText()
            row_data.append(cell_data)
        if row_data:
            df_body.append(row_data)
            
    df = pd.DataFrame(df_body, columns=header_data)
    return df

In [4]:
url = "https://wsb.gold/public/dashboard/e65fcfcb-70a4-4d86-b7fb-888057c67881"

driver.get(url)
driver.implicitly_wait(10)

## Please change this 60 seconds wait time to less if it takes less time on your PC to load page
## I introduced 60 seconds because when i tested it my net speed was slow
## and it was almost taking 30-40 seconds to load dashboard.
time.sleep(60) 

page_source = driver.page_source

flag=True
total_data = []

while flag:
    try:
        df = parse_table(page_source)
        total_data.append(df.copy)
        
        ## Find next button in table and click on it.
        element = driver.find_element_by_class_name("pr1")
        element.click()
        
        time.sleep(3) ## This is wait time after next button is click from table
        
        page_source = driver.page_source
        
    except ElementClickInterceptedException as e:
        print("Expected Exception to Stop Script : ", e)
        flag=False
    except Exception as e:
        print("Unexpected Exception : ", e)
        flag=False

Expected Exception to Stop Script :  Message: element click intercepted: Element <span class="text-brand-hover pr1 cursor-pointer disabled">...</span> is not clickable at point (356, 488). Other element would receive the click: <div class="p1 flex flex-no-shrink flex-align-right fullscreen-normal-text fullscreen-night-text">...</div>
  (Session info: headless chrome=87.0.4280.66)



In [5]:
final_df = pd.concat([data() for data in total_data])
final_df = final_df.reset_index().drop(columns=["index"])

final_df

,Count,Ticker,Market Cap
0,"21,366",GME,"10,321,160,000"
1,"11,608",AMC,"1,681,804,000"
2,"8,374",NOK,"26,522,000,000"
3,"5,356",BB,"10,544,120,000"
4,331,SLV,"14,666,550,000"
5,305,PLTR,"61,616,150,000"
6,287,BBBY,"4,469,187,000"
7,280,TSLA,"837,081,700,000"
8,273,AG,"3,091,251,000"
9,224,SPY,"308,538,600,000"
